In [3]:
import requests
import numpy as np

Set up your API token from your profile on observation portal

In [4]:
API_TOKEN = 'XXX'
PROPOSAL_ID = 'XXX'

In [23]:
# Determine when the observations are to take place
# The period is NOT the period of the variable. 
# The period is the cadence at which you want to observe, in HOURS
# The Jitter is the window within which your observation may take place. 

target_known_period = 24.72

# Want 0.035 phase intervals between each observation (this is true for all targets):
period = target_known_period*0.035 # hours
print('The observing cadence is {} mins'.format(period*60))
period = np.floor(period*60)/60
print('The observing cadence is {} mins'.format(period*60))

# Want jitter to equal period to "fully cover" time
jitter = period

cadence = {
    'start': '2019-10-05 22:00:00',
    'end': '2019-10-07 01:00:00',
    'period': period,
    'jitter': jitter
}

The observing cadence is 51.912000000000006 mins
The observing cadence is 51.0 mins


In [24]:
target = {
    'name': 'Target_C',
    'type': 'ICRS',
    'ra': 83.14124,
    'dec': -6.44638,
    'epoch': 2000
}

In [25]:
# Keep Max_airmass at 1.6
# min_lunar_distance is in DEGREES, keep at 20 degrees

constraints = {
    'max_airmass': 1.6,
    'min_lunar_distance': 20
}

In [26]:
# This does not need to be changed.
# Exposure time is in SECONDS

configurations = [
    {
        'type': 'EXPOSE',
        'instrument_type': '0M4-SCICAM-SBIG',
        'target': target,
        'constraints': constraints,
        'acquisition_config': {},
        'guiding_config': {},
        'instrument_configs': [
            {
                'exposure_time': 90,
                'exposure_count': 1,
                'optical_elements': {
                    'filter': 'v'
                }
            }
        ]
    },
    {
        'type': 'EXPOSE',
        'instrument_type': '0M4-SCICAM-SBIG',
        'target': target,
        'constraints': constraints,
        'acquisition_config': {},
        'guiding_config': {},
        'instrument_configs': [
            {
                'exposure_time': 90,
                'exposure_count': 1,
                'optical_elements': {
                    'filter': 'b'
                }
            }
        ]
    }
]

In [27]:
# This cell does not need to be edited.

# We do not provide windows for a cadence request
windows = []

# The telescope class that should be used for this observation
location = {
    'telescope_class': '0m4'
}

In [28]:
# Please input the name of your observation request. Eg. Group_Star_Target_B
requestgroup = {
    'name': 'TestData_Target_C',  # The title
    'proposal': PROPOSAL_ID,
    'ipp_value': 1.05,
    'operator': 'SINGLE',
    'observation_type': 'NORMAL',
    'requests': [{
        'cadence': cadence,
        'configurations': configurations,
        'windows': windows,
        'location': location,
    }]
}

In [29]:
response = requests.post(
    'https://observe.lco.global/api/requestgroups/cadence/',
    headers={'Authorization': 'Token {}'.format(API_TOKEN)},
    json=requestgroup  # Make sure you use json!
)

In [30]:
try:
    response.raise_for_status()
except requests.exceptions.HTTPError as exc:
    print('Request failed: {}'.format(response.content))
    raise exc

In [31]:
cadence_request = response.json()

print('Cadence generated {} requests'.format(len(cadence_request['requests'])))
i = 1
for request in cadence_request['requests']:
    print('Request {0} window start: {1} window end: {2}'.format(
        i, request['windows'][0]['start'], request['windows'][0]['end']
    ))
    i = i + 1

Cadence generated 23 requests
Request 1 window start: 2019-10-06T00:07:30Z window end: 2019-10-06T00:58:30Z
Request 2 window start: 2019-10-06T00:58:30Z window end: 2019-10-06T01:49:30Z
Request 3 window start: 2019-10-06T01:49:30Z window end: 2019-10-06T02:40:30Z
Request 4 window start: 2019-10-06T02:40:30Z window end: 2019-10-06T03:31:30Z
Request 5 window start: 2019-10-06T03:31:30Z window end: 2019-10-06T04:22:30Z
Request 6 window start: 2019-10-06T04:22:30Z window end: 2019-10-06T05:13:30Z
Request 7 window start: 2019-10-06T05:13:30Z window end: 2019-10-06T06:04:30Z
Request 8 window start: 2019-10-06T06:04:30Z window end: 2019-10-06T06:55:30Z
Request 9 window start: 2019-10-06T06:55:30Z window end: 2019-10-06T07:46:30Z
Request 10 window start: 2019-10-06T07:46:30Z window end: 2019-10-06T08:37:30Z
Request 11 window start: 2019-10-06T08:37:30Z window end: 2019-10-06T09:28:30Z
Request 12 window start: 2019-10-06T09:28:30Z window end: 2019-10-06T10:19:30Z
Request 13 window start: 2019-1

In [32]:
# Submit the fully formed RequestGroup
response = requests.post(
    'https://observe.lco.global/api/requestgroups/validate/',
    headers={'Authorization': 'Token {}'.format(API_TOKEN)},
    json=cadence_request  # Make sure you use json!
)

# Make sure the API call was successful
try:
    response.raise_for_status()
except requests.exceptions.HTTPError as exc:
    print('API call failed: {}'.format(response.content))
    raise exc

requestgroup_dict = response.json()  # The API will return the newly submitted requestgroup as json

print('Total number of observing hours needed for this request: ',
      requestgroup_dict['request_durations']['duration']/60./60)

Total number of observing hours needed for this request:  2.133888888888889


# Do not run the following cell until you have confirmed with an instructor your observation.

In [12]:
response = requests.post(
    'https://observe.lco.global/api/requestgroups/',
    headers={'Authorization': 'Token {}'.format(API_TOKEN)},
    json=cadence_request  # Make sure you use json!
)

# Make sure this API call was successful
try:
    response.raise_for_status()
except requests.exceptions.HTTPError as exc:
    print('Request failed: {}'.format(response.content))
    raise exc

requestgroup_dict = response.json()

# Print out the url on the portal where we can view the submitted request
print('View this observing request: https://observe.lco.global/requestgroups/{}/'.format(requestgroup_dict['id']))

View this observing request: https://observe.lco.global/requestgroups/858381/
